# Исследование надёжности заёмщиков
### Описание и цели
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## 1. Импорт данных

In [ ]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')


Посмотрим структуру таблицы

In [ ]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Исходя из поставленной задачи можно предположить, что для оценки влияния кол-ва детей и семейного положения на факт погашения кредита в срок для анализа нам понадобятся столбцы: 'children', 'family_status_id' где 0 - это женат/замужем, а 1 - соответствует категории не женат/не замужем (гражданский брак) и 'debt' - где 0 - отсутсвие задолженности, а 1 - задолженность есть.

In [ ]:
print(df[df['family_status_id'] == 1]['family_status_id'].count())

4177


Сводная информация по таблице

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Из сводной информации по таблице df видно, что в стобцах "days_employed" и "total_income" количество записей данных отличается от данных из других колонок.

Посмотрим есть ли в названиях столбцов лишние символы

In [ ]:
df.columns 

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Названия столбцов корректны, лишних символов нет. 

## 2. Предобработка данных

### Обработка пропусков

Проверим данные на наличие пропусков и просуммируем их количество

In [ ]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В столбцах "days_employed" и "total_income" одинаковое количество пропусков - 2174, что наводит на мысль, что пропуски не случайны и характерны для определенной категории людей "income_type". Проверим эту гипотезу.

In [ ]:
print(df['income_type'].unique())

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


Посмотрим первые 10 строк содержащих пропуски

In [ ]:
null_data = df[df.isnull().any(1)]
null_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


In [ ]:
print(null_data['income_type'].value_counts())

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64


Из отсортированной по пропускам таблице (только строки содержащие хотя бы 1 пропуск) видно, что категория людей не влияет на наличие пропусков: пропуски есть как у "пенсионер", так и у "госслужащий" и "сотрудник". Гипотеза не подтвердилась. 
Возможная причина пропусков связана с отсутствием жестких требований у банков по этим данным у заёмщиков. 

Чтобы заполнить пропущенные значения и снизить влияния этих данных на результат нужно для каждой категории людей, в строках которых есть пропуски найти медианное значение в соответствующих столбцах и заполнить ими пропуски.

Используя функцию lambda и метод fillna() заполним пропущенные значения столбца 'total_income' на соответствующие каждой категории 'income_type' медианные значения.

In [ ]:
df['total_income'] = df.groupby('income_type')['total_income'].apply(lambda x: x.fillna(x.median()))
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

Заполняем пропущенные значения в столбце 'days_employed'.
Так как этот столбец не влияет на выводы, необходимые заказчику заполняем пропущенные значения нулем.

In [ ]:
df['days_employed'] = df['days_employed'].fillna(0)
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

На этапе предобработке данных были обнаружены пропущенные значения в столбцах "days_employed" и "total_income". Вероятная гипотеза причины пропусков - отсутствие жестких требований у банка по этим данным у заёмщиков. 
Пропущенные значения в столбце "total_income" были заменены на медианные значения по каждому виду занятости.
Пропущенные значения в столбце "days_employed" были заменены на нули.

### Замена типа данных

Для более удобного чтения и дальнейшей работы нужно изменить данные с float на целочисленные значения int. 

In [ ]:
df.dtypes

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

Используем метод astype для изменения типа данных в заданных столбцах.

In [ ]:
df[['days_employed', 'total_income']] = df[['days_employed', 'total_income']].astype('int')

#проверим изменения
df.dtypes

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

In [ ]:
df.loc[:, ['days_employed', 'total_income']].head()

,days_employed,total_income
0,-8437,253875
1,-4024,112080
2,-5623,145885
3,-4124,267628
4,340266,158616


**Вывод**

Были изменены типы данных с float на int. Для изменения типа данных в заданных столбцах был использован метод astype().

### Обработка дубликатов

Для точного определения дубликатов нужно привести все записи object в соответствующих столбцах привести к нижнем регистру и применить метод duplicated().sum() для подсчета количества дубликатов в таблице.

In [ ]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [ ]:
df[['education', 'family_status', 'gender', 'income_type', 'purpose']] = df[['education', 'family_status', 'gender', 'income_type', 'purpose']].apply(lambda x: x.str.lower())
df[['education', 'family_status', 'gender', 'income_type', 'purpose']].head()

,education,family_status,gender,income_type,purpose
0,высшее,женат / замужем,f,сотрудник,покупка жилья
1,среднее,женат / замужем,f,сотрудник,приобретение автомобиля
2,среднее,женат / замужем,m,сотрудник,покупка жилья
3,среднее,женат / замужем,m,сотрудник,дополнительное образование
4,среднее,гражданский брак,f,пенсионер,сыграть свадьбу


In [ ]:
df.duplicated().sum()

71

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

#### Обработка случайных ошибок в данных.

В данных могут содержаться случайные ошибки, связанные с неправильным вводом информации или технические ошибки при выгрузке данных. Для начала их нужно найти. 
По первым строкам таблицы было видно, что данные стобца 'days_employed' имеют, как отрицательные, так и положительные значения. 

In [ ]:
# приведем все значения столбца 'days_employed' в положительные методом abs()
df['days_employed'] = df['days_employed'].abs()
df['days_employed'].head()


0      8437
1      4024
2      5623
3      4124
4    340266
Name: days_employed, dtype: int64

Посмотрим уникальные значеничения в столбце 'children'

In [ ]:
df['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Из сводки по уникальным значениям столбца 'children' видно,  значения -1 и 20 вылетают и не вписываются в логику: количество людей уменьшается с увеличением количества детей. Вероятнее всего в записи данных были допущены ошибки и под значением 20 подразумевалось 2, а -1 сделаем положительным.

In [ ]:
df['children'] = df['children'].abs()
df['children'] = df['children'].replace({20 : 2})
df['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

**Вывод**

Все записи object были приведены к нижнему регистру. После было обнаружен 71 дубликат. Все повторяющиеся строки были удалены и обновлена индексация. 
Дубликаты могли появится вследствии сбоя в записи данных. Также были обнарежены и устранены случайные ошибки, которые могли исказить результаты.

### Лемматизация

Выделим леммы в значениях столбца с целями получения кредита "purpose".
Для этого сначала импортируем библиотеку pymystem3

In [ ]:
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

Для того, чтобы лематизировать каждую ячейку строки нужно составить фукнцию, которая будет принимать значения из соответствующей ячейки, а на выходу отдавать лематизированные слова. 

Чтобы не лемматизировать одинаковые значения сначала создадим dataframe с уникальными значениями столбца "purpose". Далее к каждому уникальному значению применим лемматизацию и результат сохраним в новом столбце. Далее используем sum(), чтобы привести все лемматизированные слова в один список.

In [ ]:
purpose_unique = df['purpose'].drop_duplicates().to_frame()
purpose_unique['lemmas'] = purpose_unique['purpose'].apply(m.lemmatize)
purpose_lemmas = purpose_unique['lemmas'].sum()

Применим метод Counter к списку для того, чтобы потом создать словарь для дальнейшей категоризации по целям кредита


In [ ]:
print(Counter(purpose_lemmas))

# Список с категориями по целям кредита
dict_purpose_category = ['недвижимость','автомобиль', 'образование', 'свадьба', 'ремонт']

Counter({' ': 59, '\n': 38, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'подержать': 1, 'со': 1, 'подержанный': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1})


Далее нам нужно написать функцию, которая как аргумент принимает значение столбца 'purpose', а возвращает название категории из словаря "purpose_category". Так как в некоторых значениях 'purpose' встречаются несколько слов из словаря, например, "ремонт недвижимости" и слова синонимы, например, "жилье - недвижимость" то для таких значений отдельно запишем условия, которое будет указывать на верную категорию по целям кредита. Так было оперделено, что "ремонт недвижимости" будет в категории "ремонт", потому что это совсем другая цель по сумме кредита относительно приобретения недвижимости. А все остальные цели связанные с недвижимостью будут относится к категории покупке жилья - коротко "недвижимость".

In [ ]:
  
def check_purpose(purpose):
    lemmas = m.lemmatize(purpose)
    for element in dict_purpose_category:
        if 'ремонт' in lemmas:
            return 'ремонт'
        if element in lemmas:
            return element
        if 'жилье' in lemmas:
            return 'недвижимость'
        
            
    return 'другое'

**Вывод**

Выделили леммы в значениях столбца "purpose" (цель получения кредита). На основании списка с количеством лемматизированных слов определили словарь категорий по цели получения кредита. Написали функцию, которая как аргумент принимает значение столбца 'purpose', а возвращает название категории из словаря "purpose_category.

### Категоризация данных

Для того, что было легче интерпретировать данные и объеденять их в сводные таблицы создали несколько категорий, объединяющих данные по определенному признаку. 

Вот какие категории были добавлены в таблицу: по целям кредита, по уровню дохода, наличие / отсутствие задолженности, наличие / отсутствие детей.

#### Категории по целям кредита

In [ ]:
# добавим в нашу исходную таблицу столбец с категорией по целям кредита, используя ранее написанную функцию check_purpose
df['purpose_category'] = df['purpose'].apply(check_purpose)
df['purpose_category'].value_counts()

недвижимость    10204
автомобиль       4306
образование      4013
свадьба          2324
ремонт            607
Name: purpose_category, dtype: int64

#### Категории по уровню дохода

С помощью метода discribe() данные столбца 'total_income' были разделены на 4 категории по уровню дохода: "низкий" от 20667 до 107623, "ниже среднего" от 107623 до 142594, "выше среднего" от 142594 до 195820, "высокий" от 195820 до 2265604.  

In [ ]:
#посмотрим основные статистические показатели 
df['total_income'].describe()

count    2.145400e+04
mean     1.653196e+05
std      9.818730e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.425940e+05
75%      1.958202e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [ ]:
# напишем функцию, которая возвращает название категории по значению столбца общий доход 
def check_incom(income):
    if income <= 2265604 and income > 195820:
        return 'высокий'
    if income <= 195820 and income > 142594:
        return 'выше среднего'
    if income <= 142594 and income > 107623:
        return 'ниже среднего'
    return 'низкий'

In [ ]:
# добавим новый столбец в нашу исходную таблицу с категориями по доходу.
df['category_income'] = df['total_income'].apply(check_incom)
df['category_income'].value_counts()

ниже среднего    5479
высокий          5364
низкий           5364
выше среднего    5247
Name: category_income, dtype: int64

#### Категории по наличию детей

Составили простую функцию, которая принимает значение столбца "children", а возвращает наименование категории "дети есть" или "детей нет". Записали работу этой функции в новый столбец "having_children". 

In [ ]:
def check_children(children):
    if children > 0:
        return 'дети есть'
    return 'детей нет'

In [ ]:
df['having_children'] = df['children'].apply(check_children)
df['having_children'].value_counts()

детей нет    14091
дети есть     7363
Name: having_children, dtype: int64

#### Категории по наличию задолженности

Здесь добавили новый столбец "debt_category", куда записывали "долг есть", если в значении стобца "debt" есть 1 и "дога нет", если в значении есть 0.

In [ ]:
df['debt_category'] = df['debt'].replace({1 : 'долг есть', 0 : 'долга нет'})
df['debt_category'].value_counts()

долга нет    19713
долг есть     1741
Name: debt_category, dtype: int64

Проверим сумму столбца "debt" (количество должников), чтобы сранить с полученным выше результатом.

In [ ]:
df['debt'].sum()

1741

**Вывод**

На основе обработанных данных в исходную таблицу были добавлены следующие категории: по целям кредита, по уровню дохода, наличие / отсутствие задолженности, наличие / отсутствие детей. Теперь таблица готова к выводу результатов на основе сводных таблиц по сформированным категориям.

## 3. Исследование и выводы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Составили сводную таблицу по категориям наличия детей относительно наличия / отсутствия долга.

In [ ]:
children_pivot_table = df.pivot_table(index='having_children', columns='debt_category', values='debt', aggfunc='count')
children_pivot_table['долг есть, %'] = children_pivot_table['долг есть'].div(children_pivot_table.sum(axis=1), axis=0).multiply(100).astype('int')
children_pivot_table['долга нет, %'] = 100 - children_pivot_table['долг есть, %']
children_pivot_table

debt_category,долг есть,долга нет,"долг есть, %","долга нет, %"
having_children,,,,
детей нет,1063,13028,7,93
дети есть,678,6685,9,91


**Вывод**

По результатам анализа сводной таблицы было установлено, что люди, у которых есть дети на 2 % реже возвращают кредит в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
family_status_pivot_table = df.pivot_table(index='family_status', columns='debt_category', values='debt', aggfunc='count')
family_status_pivot_table['долг есть, %'] = family_status_pivot_table['долг есть'].div(family_status_pivot_table.sum(axis=1), axis=0).multiply(100).astype('int')
family_status_pivot_table['долга нет, %'] = 100 - family_status_pivot_table['долг есть, %']
family_status_pivot_table

debt_category,долг есть,долга нет,"долг есть, %","долга нет, %"
family_status,,,,
в разводе,85,1110,7,93
вдовец / вдова,63,896,6,94
гражданский брак,388,3763,9,91
женат / замужем,931,11408,7,93
не женат / не замужем,274,2536,9,91


**Вывод**

По результатам анализа сводной таблицы было установлено, что люди, которые не состоят в браке на 2 % меньше возвращают кредит в срок относительно людей в браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:

level_income_pivot_table = df.pivot_table(index='category_income', columns='debt_category', values='debt', aggfunc='count')
level_income_pivot_table['долг есть, %'] = level_income_pivot_table['долг есть'].div(level_income_pivot_table.sum(axis=1), axis=0).multiply(100).astype('int')
level_income_pivot_table['долга нет, %'] = 100 - level_income_pivot_table['долг есть, %']
level_income_pivot_table

debt_category,долг есть,долга нет,"долг есть, %","долга нет, %"
category_income,,,,
высокий,383,4981,7,93
выше среднего,448,4799,8,92
ниже среднего,483,4996,8,92
низкий,427,4937,7,93


**Вывод**

Удивительно, но уровень дохода не влияет на возврат кредита в срок.

- Как разные цели кредита влияют на его возврат в срок?

Составили сводную таблицу по категориям целей кредита относительно наличия / отсутствия долга.

In [ ]:
purpose_pivot_table = df.pivot_table(index='purpose_category', columns='debt_category', values='debt', aggfunc='count')
purpose_pivot_table['долг есть, %'] = purpose_pivot_table['долг есть'].div(purpose_pivot_table.sum(axis=1), axis=0).multiply(100).astype('int')
purpose_pivot_table['долга нет, %'] = 100 - purpose_pivot_table['долг есть, %']
purpose_pivot_table

debt_category,долг есть,долга нет,"долг есть, %","долга нет, %"
purpose_category,,,,
автомобиль,403,3903,9,91
недвижимость,747,9457,7,93
образование,370,3643,9,91
ремонт,35,572,5,95
свадьба,186,2138,8,92


**Вывод**

7 % людей не вернули кредит в срок в категории "недвижимость". В то время, как на образование и автомобиль 9 % людей не вернули кредит в срок. Вероятно, это связано с тем, что к большой покупке квартиры люди финансово подготавливаются более основательно, чем на краткосрочные цели, такие как автомобиль и образование, свадьба.

## 4. Выводы

На основе предоставленных данных статистики о платежеспособности клиентов были сделаны следующие выводы:
1. В данных были обнаружены дубликаты и артефакты, что говорит о том, что возможно были допущены технические ошибки на которые следует обратить внимание при следующих сборах информациии.
2. Уровень дохода практически не влияет на возврат кредита в срок.
3. Большие цели кредита, такие как покупка недвижимости, увеличивает вероятность возврата кредита в срок, чем краткосрочные: покупка машины, свадьба или образование.
4. Люди, которые состоят в браке возвращают кредит в срок на 2 % больше, чем те, которые не состоят в браке.